# Unified Plotting

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#pi0scaling = {"val": 0, "suffix": "", "title": "No pi0 Scaling"} #no scaling
#pi0scaling = {"val": 1, "suffix": "_pi0flat", "title": "0.759 pi0 Scaling"} #flat scaling
pi0scaling = {"val": 2, "suffix": "_pi0e035", "title": "Energy-dependent pi0 Scaling"} #energy-dependent


In [ ]:
from matplotlib import pyplot as plt

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
plt.rcParams.update(params)

from load_data_run123 import *
from unblinding_far_sideband import *

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import scipy.stats

In [ ]:
#which_sideband = ['np_far'       ,"farsideband"] # far-sideband 1eNp datasets
#which_sideband = ['2plus_showers',"farsideband"] # far-sideband 2+ shower datasets
which_sideband = ['opendata'     ,'opendata']    # open datasets (5e19 / 1e19)

In [ ]:
samples = load_data_run123(which_sideband=which_sideband[0], return_plotter=False, pi0scaling=pi0scaling["val"])


In [ ]:
import plotter
import unblinding_far_sideband
import importlib
importlib.reload(plotter)
importlib.reload(unblinding_far_sideband)

In [ ]:
plotter_runbyrun = {}
for run in [1, 2, 3, 123, 12]:
    this_weights, this_pot = get_weights(run,dataset=which_sideband[1])
    plotter_runbyrun[run] = plotter.Plotter(samples, this_weights, pot=this_pot)

In [ ]:
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

pvaluetypes = ['pvalue','pvaluediag','pvaluestatonly']

# Single Variable Plotter

In [ ]:
from unblinding_far_sideband import *

# choose sideband, preselection, and selection
# find defintions in unblinding_far_sideband.py
sdb_key = 'None'
pre_key = 'NP'
sel_key = 'NPVL'
run = 123

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

QUERY = ""
for query in [sideband['query'],preselection['query'],sel['query']]:
    if query is None: continue
    if QUERY: QUERY += ' and '
    QUERY += query
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'

#formatting can be improved..
TITLE = ""
for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"]]:
    if text is None: continue
    TITLE += text

#VARIABLE, BINS, RANGE, XTIT = 'topological_score',20,(0,1),r"topological score"
VARIABLE, BINS, RANGE, XTIT = 'reco_e',20,(0,2),r"reco energy"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_score',10,(0,1),r"pi0 score"
#VARIABLE, BINS, RANGE, XTIT = 'n_showers_contained',10,(-0.5, 9.5),"n showers contained"
print(VARIABLE, BINS, RANGE, XTIT)

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARIABLE,   
    query=QUERY,
    kind="event_category",
    draw_sys=True,
    stacksort=3,
    title=XTIT,
    bins=BINS,
    range=RANGE,
)[0:3]


ax1.set_ylim(0, ax1.get_ylim()[1]*1.8)

ax1.set_title(TITLE, loc='left')

plt.tight_layout()
fig.show()
#save_path = this_folder+VARIABLE
#fig.savefig(save_path + '.png', dpi=250)    
#plt.close()
 

In [ ]:
#samples['ext'].query(unblinding_far_sideband.BDTCQ)[['run','evt','reco_e']]
samples['ext'].query(unblinding_far_sideband.BDTCQ)[['run','evt','reco_e','CosmicIPAll3D','trkpid','hits_ratio','trkfit','n_showers_contained','tksh_distance','tksh_angle']]

# Multiple variable plotting

In [ ]:
from unblinding_far_sideband import *

# choose sideband, preselection, and selection(s)
# find defintions in unblinding_far_sideband.py
sdb_key = 'None'
pre_key = 'NP'
sel_keys = ['None','NPVL']

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]

for run in [1]: #[1,2,3,12,123]:
    for sel_key in sel_keys:
        sel =  selection_categories[sel_key]
        print('\nRun %i, %s, %s, %s'%(run, sideband['dir'], preselection['dir'], sel['dir']))
        if (which_sideband[1] != "opendata"):
            this_folder = ls.plots_path+'PELEE_far_sideband_runbyrun/'
        else:
            this_folder = ls.plots_path+'PELEE_opendata/'
        this_folder += 'run_{}/'.format(run)
        this_folder += '{}_{}_{}'.format(sideband['dir'], preselection['dir'], sel['dir'])
        this_folder += (pi0scaling["suffix"]+'/')
        !mkdir -p $this_folder

        this_query = ""
        for query in [sideband['query'],preselection['query'],sel['query']]:
            if query is None: continue
            if this_query: this_query += ' and '
            this_query += query
        if run in [1, 2, 3, 12]:
            this_query += f' and run{run}==True'

        #formatting can be improved..
        this_title = ""
        for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"]]:
            if text is None: continue
            this_title += text

        pvalues = [[] for _ in range(len(pvaluetypes))]
        
        for plot_variable in plot_variables:
            VARIABLE, BINS, RANGE, XTIT = plot_variable[0:4]
            #if VARIABLE != 'tksh_angle': continue
            #if len(plot_variable)>4 or VARIABLE not in TRAINVAR: continue
            print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=True,
                stacksort=3,
                title=XTIT,
                bins=BINS,
                range=RANGE,
            )[0:3]

            if len(plot_variable) == 6:
                if plot_variable[5] is True:
                    ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.8)

            ax1.set_title(this_title, loc='left')

            plt.tight_layout()
            save_path = this_folder+VARIABLE
            if len(plot_variable) >= 5:
                save_path += ('_' + plot_variable[4])
            fig.savefig(save_path + '.png', dpi=250)    
            plt.close()
            # store pvalues
            if len(plot_variable)==4 and VARIABLE in TRAINVAR:
                for idx, pvt in enumerate(pvaluetypes):
                    pvalues[idx].append(plotter_runbyrun[run].stats[pvt])
            #print ('%s %.04f'%(VARIABLE,plotter_runbyrun[run].stats['chisqstatonly']))
            print('%s %.4f %.4f %.4f' % (VARIABLE, plotter_runbyrun[run].stats['pvaluestatonly'], plotter_runbyrun[run].stats['pvaluediag'], plotter_runbyrun[run].stats['pvalue']))
        # make pvalue plots
        #print(pvalues)
        for idx, pvt in enumerate(pvaluetypes):
            if len(pvalues[idx])!=16: continue
            plt.figure(figsize=(9, 7))
            plt.hist(pvalues[idx], bins=10, range=(0,1))
            plt.title(this_title, loc='left')
            plt.xlabel(pvt)
            plt.ylabel("number of BDT input variables")
            plt.savefig(this_folder + pvt + '.png', dpi=250)
            plt.close()

In [ ]:
samples['data'].query(BDTCQ)[["run","sub","evt","reco_e","pi0_score","nonpi0_score","n_tracks_contained","reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z"]]